In [80]:
from os import rename
from os.path import split, join
from glob import glob
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from itertools import product
from IPython.display import display, HTML
pd.options.display.max_columns = 100
pd.options.display.max_rows = 10
sns.set_context("paper", font_scale=1.4)

# FHHPS Empirical Application

This reports the result of applying our method to Allcott's dataset, which contained a bit over 10k observations.

## Tuning parameter selection

In the same notation of the tuning parameters as in the `simulation_figures.ipynb`, we drew over 150k tuning parameters from the grid below.

+ $c_{shocks} \in [.01, 5]$
+ $c_{output1\_step1} \in [0.01, 5]$
+ $c_{output1\_step2} \in [0.01, 5]$
+ $c_{output2} \in [0.01, 5]$
+ $c_{censor1} \in [0.01, 2]$
+ $c_{censor2} \in [0.01, 2]$
+ kernel $\in$ {KNN, neighbor}

Note that the asymptotic results in the paper do not provide guide explicit guidance on how to choose these parameters.

With a lot of data, these choices matter less. But as we will see below, at 10k observation they still matter a great deal.

## Reading in

Reading in the results. Ignore this section

In [82]:
dfs = []
for k, file in enumerate(glob(join("empirical_out", "*43cd*"))):
    df_tmp = pd.read_csv(file, header=None)
    dfs.append(df_tmp)
    
df = pd.concat(dfs).dropna()

mean_names = ["EA", "EB", "EC"]
cov_names = ["VarA", "VarB", "VarC", "CovAB", "CovAC", "CovBC"]
param_names = ["shock_bw1_const", "output_bw1_coqnst_step1", "output_bw1_const_step2", "output_bw2_const"]
pretty_param_names = ["$c_{shock}$", "$c_{y,1}^{(1)}$", "$c_{y,1}^{(2)}$", "$c_{y,2}$"]

params = ['n', 'kernel1', 'kernel2', 
      'output_bw1_const_step1', 'output_bw1_const_step2', 'output_bw2_const',
      'output_bw1_alpha', 'output_bw2_alpha', 
      'shock_bw1_const', 'shock_bw2_const', 'shock_bw1_alpha', 'shock_bw2_alpha', 
      'censor1_const', 'censor2_const']
others = ['mean_valid', 'cov_valid',"time"]
cols = params + others
df.columns = cols + mean_names + cov_names
df = df.drop_duplicates(params)

In [83]:
f"We tried {len(df)} different tuning parameter combinations"

'We tried 14054 different tuning parameter combinations'

In [84]:
df.head()

,n,kernel1,kernel2,output_bw1_const_step1,output_bw1_const_step2,output_bw2_const,output_bw1_alpha,output_bw2_alpha,shock_bw1_const,shock_bw2_const,shock_bw1_alpha,shock_bw2_alpha,censor1_const,censor2_const,mean_valid,cov_valid,time,EA,EB,EC,VarA,VarB,VarC,CovAB,CovAC,CovBC
0,9395,neighbor,neighbor,4.500400,4.346707,1.528788,0.1,0.1,3.757622,2.853055,0.166667,0.166667,2.0,2.0,0.531559,0.058648,178.836059,13.524115,0.140234,0.124516,2026.616073,5.803292,1.904936,-92.852966,-17.865824,-0.703976
1,9395,gaussian,neighbor,4.834289,3.977185,3.508628,0.1,0.1,4.459393,4.564068,0.166667,0.166667,1.0,2.0,0.694412,0.058648,188.947886,13.213327,0.159177,0.124063,-2888.893322,-8.059475,0.788845,157.364318,-7.076770,-0.365712
2,9395,gaussian,gaussian,1.286407,0.104195,4.896878,0.1,0.1,1.482180,3.630251,0.166667,0.166667,1.0,1.0,0.694412,0.071847,165.047976,10.599901,0.149263,0.299468,2955.443143,5.143455,10.094469,-50.768644,-125.034240,-2.162901
3,9395,neighbor,gaussian,1.395385,2.323063,0.748164,0.1,0.1,3.330909,3.552299,0.166667,0.166667,0.5,0.5,0.821927,0.088558,173.477614,10.580090,0.148119,0.303441,1357.859786,2.934821,4.464864,-30.236328,-43.834269,-1.573698
4,9395,neighbor,gaussian,1.426092,0.707818,4.654991,0.1,0.1,3.865914,1.135708,0.166667,0.166667,1.0,1.0,0.694412,0.071847,175.212515,10.658341,0.149470,0.295128,2219.474602,5.781875,3.014913,-90.536651,-31.356185,-0.931089


## Results

The estimates are **extremely** sensitive to tuning parameters. For example, here is the range of interecept estimates.

In [85]:
df["EA"].agg(["min", "max"])

min     5.670008
max    20.305917
Name: EA, dtype: float64

And here's the range of the variance of the first slope, for another example.

In [86]:
df["VarB"].agg(["min", "max"])

min       -29.376151
max    807105.707823
Name: VarB, dtype: float64

## Restricting the grid of tuning parameters

Some choices of tuning parameters will give entirely unreasonable estimates.

So let's say that we want to constraint out tuning parameter selection by discard any configuration that produces mathematically impossible numbers, such as:

+ Negative variance estimates
+ Correlations larger than 1

Also, let's discard configuration that produce estimates that don't make economic sense. We'll restrict to parameters that give us:

+ $E[A_{1}] \geq 0$
+ $E[B_{1}] \geq -0.5$
+ $E[C_{1}] \geq -0.5$
+ $Var[A_{1}] < 50$
+ $Var[B_{1}] < 20$
+ $Var[C_{1}] < 20$
+ $|Corr[A_{1}, B_{1}]| < 1$ (similar for other correlations)


If we impose all of these restrictions, what are we left with?

In [87]:
df_good = df[
    
    # Variances are positive
    (df["VarA"] > 0) & (df["VarB"] > 0)  & (df["VarC"] > 0) 
    
    # Variances have reasonable magnitude
    & (df["VarA"] < 50) & (np.abs(df["CovAB"]) < 20)  & (np.abs(df["CovAC"]) < 20) 

    # Correlations are at most 1 in absolute value
    & (np.abs(df["CovAC"]) < np.sqrt(df["VarA"]*df["VarC"]))
    & (np.abs(df["CovAB"]) < np.sqrt(df["VarA"]*df["VarB"]))
    & (np.abs(df["CovBC"]) < np.sqrt(df["VarB"]*df["VarC"]))
    
    # Sensible average values
    & (df["EA"] > 0) & (df["EB"] > -.5) & (df["EC"] > -.5)
    
].drop_duplicates()

/Users/vitorh/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in sqrt
  from ipykernel import kernelapp as app


Restricted dataset (19 configurations, out of the 150k+ we started with.) 

In [88]:
df_good

,n,kernel1,kernel2,output_bw1_const_step1,output_bw1_const_step2,output_bw2_const,output_bw1_alpha,output_bw2_alpha,shock_bw1_const,shock_bw2_const,shock_bw1_alpha,shock_bw2_alpha,censor1_const,censor2_const,mean_valid,cov_valid,time,EA,EB,EC,VarA,VarB,VarC,CovAB,CovAC,CovBC
6,9395,neighbor,gaussian,4.371484,2.866751,0.137582,0.1,0.1,4.163132,1.438553,0.166667,0.166667,2.0,0.5,0.531559,0.088558,162.361472,10.786974,0.132309,0.303826,36.97974,1.680448,2.104846,-0.900945,-7.846881,-1.356938


**Conclusion: Even after restricting to 'sensible' tuning parameters, we still observe a lot of variation in our point estimates**

In [89]:
df_good[["EA", "EB", "EC"]].describe()

,EA,EB,EC
count,1.000000,1.000000,1.000000
mean,10.786974,0.132309,0.303826
std,NaN,NaN,NaN
min,10.786974,0.132309,0.303826
25%,10.786974,0.132309,0.303826
50%,10.786974,0.132309,0.303826
75%,10.786974,0.132309,0.303826
max,10.786974,0.132309,0.303826


In [90]:
df_good[["VarA", "VarB", "VarC", "CovAB", "CovAC", "CovBC"]].describe()

,VarA,VarB,VarC,CovAB,CovAC,CovBC
count,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000
mean,36.97974,1.680448,2.104846,-0.900945,-7.846881,-1.356938
std,NaN,NaN,NaN,NaN,NaN,NaN
min,36.97974,1.680448,2.104846,-0.900945,-7.846881,-1.356938
25%,36.97974,1.680448,2.104846,-0.900945,-7.846881,-1.356938
50%,36.97974,1.680448,2.104846,-0.900945,-7.846881,-1.356938
75%,36.97974,1.680448,2.104846,-0.900945,-7.846881,-1.356938
max,36.97974,1.680448,2.104846,-0.900945,-7.846881,-1.356938


As we can see above, we can get estimates of (e.g.) $Var[A_{1}]$ as low as 24 and as high as 49, depending on the parameters.

## Can we get 'better' estimates?

If we don't "like" these estimates, then by fiddiling with the tuning parameters we can get "better" estimates. 

For example, now that we know that we have a better idea about which parameters yield more sensible estimates, we can keep searching on a finer grid in that region until we get estimates that make economic sense to us.

But would we want to do that? (No)